# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
#from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
#import wget

# Crawler

In [29]:
Dominio = ['https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://www.tcgplayer.com/','https://scryfall.com/','http://www.mtgmintcard.com/']
Dados = []
TempoDominio = []

In [30]:
# Função para marcar a page como visitada.
def Visitou (url, paginasVisitadas):
    if url not in paginasVisitadas:
        return False
    else:
        return True

In [31]:
#Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    try:
        response = rq.get(url.geturl())
        if response.status_code == rq.codes.ok:
            return True
        else:
            return False
    except:
        return False

In [32]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [33]:
#Função para Verificar se podemos baixar a page ou não
def verificarRobotTxt(url):
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [34]:
# Criar uma Tabela com as informações sobre links.
def Tabela(count,url,site):
    Dados.append(dict(zip(['Número','Link','Site'],[count,url,site])))

## * Baseline

In [38]:
def CardsCrawler(url):
    print('Dominio: ' + url.geturl())
    count = 0
    paginas = [url]
    paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        primeiraPagina = paginas.pop(0)
        # Verifica se a pagina foi visitada.Se foi, procuro o proximo link da lista que não foi visitado.
        # Verifico se o hostname de cada pagina é igual ao do dominio, se não, rocuro o proximo link da lista (navegar apenas dentro do site)
        # Verifico se hostname existe , se não , procuro o proximo link que tenha.
        # Verifico se é possivel crawlear a pagina de acordo com as regras do robot.txt , se não , procuro o proximo link.
        # verificar se a requisição funciona.
        while Visitou(primeiraPagina.geturl(), paginasVisitadas) == True or url.hostname == None or primeiraPagina.hostname != url.hostname or robots.allowed(primeiraPagina.geturl(), '*') == False or verificarRequest(primeiraPagina) == False:
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
        
        # Verficar se o site é dinamico.
        if (Dominio[3] in url.geturl()) or (Dominio[6] in url.geturl()):
            texto = CrawlerDinâmico(primeiraPagina.geturl())
        else:
            #request - usando o content type
            codigo_fonte = rq.get(primeiraPagina.geturl(),headers={'content-type': 'text/html'})

            # texto do codigo_fonte.
            texto = codigo_fonte.text
                
        soup = BeautifulSoup(texto,'html.parser')

        # Encontrar todos os links da pagina(url). # attrs={'href': re.compile("^http://")}
        for link in soup.findAll('a'):
            href = link.get('href')
            # Correção de href relativos para absolutos.
            href = urllib.parse.urljoin(url.geturl(),href)
            paginas.append(urlparse(href))

        # Marco como visitada a urlparse    
        paginasVisitadas.append(primeiraPagina.geturl())
       
        #esperar 500 milisegundos para visitar o prox link.
        time.sleep(.5)
        
        count = count + 1
        print(count,'-',primeiraPagina.geturl(),' Qtd Paginas=', len(paginas))

        # Salvar a page localmente
        with open("Paginas/PaginasBaseline/"+str(url.hostname)+str(count)+".html", 'w' , encoding='utf-8') as outfile:
            outfile.write(texto)
        
        #urllib.request.urlretrieve(primeiraPagina.geturl(), "Paginas/PaginasBaseline/"+str(url.hostname)+str(count)+".html")
        #wget.download(primeiraPagina.geturl(), "Paginas/PaginasBaseline/"+str(url.hostname)+str(count)+".html")

        Tabela(count,primeiraPagina.geturl(),url.hostname)

In [39]:
Dominio

['https://www.capefeargames.com/',
 'https://www.mtgotraders.com/',
 'https://www.wizardscupboard.com/',
 'https://abugames.com/',
 'https://www.cardkingdom.com/',
 'http://www.starcitygames.com/',
 'https://www.tcgplayer.com/',
 'https://scryfall.com/',
 'http://www.mtgmintcard.com/']

### Download dos 8 primeiros sites escolhidos pro Dominio
* Falha no tcgplayer

In [12]:
for url in Dominio:
    inicio = time.time()
    CardsCrawler(urlparse(url))
    fim = time.time()
    tempo = fim - inicio
    print('Tempo do Site: ',tempo, 'Site: ', url)
    TempoDominio.append(dict(zip(['Dominio','Tempo'],[url,tempo])))
    

Dominio: https://www.capefeargames.com/
1 - https://www.capefeargames.com/  Qtd Paginas= 872
2 - https://www.capefeargames.com/buylist  Qtd Paginas= 1633
3 - https://www.capefeargames.com/faq__policies  Qtd Paginas= 2478
4 - https://www.capefeargames.com/specials  Qtd Paginas= 3390
5 - https://www.capefeargames.com/products/multi_search  Qtd Paginas= 4232
6 - https://www.capefeargames.com/retail_store  Qtd Paginas= 5077
7 - https://www.capefeargames.com/user/login  Qtd Paginas= 5921
8 - https://www.capefeargames.com/advanced_search  Qtd Paginas= 6763
9 - https://www.capefeargames.com/user/signup  Qtd Paginas= 7598
10 - https://www.capefeargames.com/catalog/magic_singles/8  Qtd Paginas= 8489
11 - https://www.capefeargames.com/catalog/event_registration-magic_events-prerelease_events/4277  Qtd Paginas= 9369
12 - https://www.capefeargames.com/products/search?q=commander+2019  Qtd Paginas= 10329
13 - https://www.capefeargames.com/catalog/magic_singles-throne_of_eldraine/oko_thief_of_crowns

78 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2015/1453  Qtd Paginas= 70580
79 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2014/871  Qtd Paginas= 71566
80 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2013/774  Qtd Paginas= 72552
81 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2012/702  Qtd Paginas= 73540
82 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2011/298  Qtd Paginas= 74532
83 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2010/224  Qtd Paginas= 75519
84 - https://www.capefeargames.com/catalog/magic_singles-core_sets-10th_edition/76  Qtd Paginas= 76506
85 - https://www.capefeargames.com/catalog/magic_singles-core_sets-9th_edition/75  Qtd Paginas= 77491
86 - https://www.capefeargames.com/catalog/magic_singles-core_sets-8th_edition/74  Qtd Paginas= 78477
87 - https://www.capefeargames.com/catalog/magic_singles-core_sets-7th_edition/7

155 - https://www.capefeargames.com/catalog/magic_singles-invasion_block-invasion/42  Qtd Paginas= 144479
156 - https://www.capefeargames.com/catalog/magic_singles-invasion_block-planeshift/41  Qtd Paginas= 145463
157 - https://www.capefeargames.com/catalog/magic_singles-invasion_block-apocalypse/40  Qtd Paginas= 146448
158 - https://www.capefeargames.com/catalog/magic_singles-masques_block/43  Qtd Paginas= 147295
159 - https://www.capefeargames.com/catalog/magic_singles-masques_block-mercadian_masques/46  Qtd Paginas= 148281
160 - https://www.capefeargames.com/catalog/magic_singles-masques_block-nemesis/45  Qtd Paginas= 149265
161 - https://www.capefeargames.com/catalog/magic_singles-masques_block-prophecy/44  Qtd Paginas= 150249
162 - https://www.capefeargames.com/catalog/magic_singles-saga_block/47  Qtd Paginas= 151096
163 - https://www.capefeargames.com/catalog/magic_singles-saga_block-urzas_saga/50  Qtd Paginas= 152100
164 - https://www.capefeargames.com/catalog/magic_singles-saga

228 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault/294  Qtd Paginas= 215780
229 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_angels/2583  Qtd Paginas= 216728
230 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_annihilation/1543  Qtd Paginas= 217676
231 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_dragons/622  Qtd Paginas= 218621
232 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_exiled/296  Qtd Paginas= 219567
233 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_legends/714  Qtd Paginas= 220510
234 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_lore/3133  Qtd Paginas= 221463
235 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_realms/798  Qtd Paginas= 222410
236 - https://www.capefeargames.com/catalog/magic_sin

296 - https://www.capefeargames.com/catalog/magic_singles-signature_spellbook_series/4013  Qtd Paginas= 280571
297 - https://www.capefeargames.com/catalog/magic_singles-signature_spellbook_series-signature_spellbooks_jace/3983  Qtd Paginas= 281525
298 - https://www.capefeargames.com/catalog/magic_singles-signature_spellbook_series-signature_spellbook_gideon/4286  Qtd Paginas= 282475
299 - https://www.capefeargames.com/catalog/magic_singles-special_editions/77  Qtd Paginas= 283335
300 - https://www.capefeargames.com/catalog/magic_singles-special_editions-anthologies/697  Qtd Paginas= 284328
301 - https://www.capefeargames.com/catalog/magic_singles-special_editions-battlebond/4003  Qtd Paginas= 285314
302 - https://www.capefeargames.com/catalog/magic_singles-special_editions-battle_royal/85  Qtd Paginas= 286300
303 - https://www.capefeargames.com/catalog/magic_singles-special_editions-beatdown/84  Qtd Paginas= 287296
304 - https://www.capefeargames.com/catalog/magic_singles-special_editi

...

334 - https://scryfall.com/sets/pdom/ja  Qtd Paginas= 65480
335 - https://scryfall.com/sets/tdom  Qtd Paginas= 65549
336 - https://scryfall.com/sets/ddu  Qtd Paginas= 65675
337 - https://scryfall.com/sets/ddu/ja  Qtd Paginas= 65800
338 - https://scryfall.com/sets/tddu  Qtd Paginas= 65857
339 - https://scryfall.com/sets/a25  Qtd Paginas= 66156
340 - https://scryfall.com/sets/a25/ja  Qtd Paginas= 66455
341 - https://scryfall.com/sets/a25/zhs  Qtd Paginas= 66757
342 - https://scryfall.com/sets/ta25  Qtd Paginas= 66826
343 - https://scryfall.com/sets/plny  Qtd Paginas= 66877
344 - https://scryfall.com/sets/pnat  Qtd Paginas= 66928
345 - https://scryfall.com/sets/rix  Qtd Paginas= 67183
346 - https://scryfall.com/sets/rix/es  Qtd Paginas= 67438
347 - https://scryfall.com/sets/rix/fr  Qtd Paginas= 67696
348 - https://scryfall.com/sets/rix/de  Qtd Paginas= 67954
349 - https://scryfall.com/sets/rix/it  Qtd Paginas= 68212
350 - https://scryfall.com/sets/rix/pt  Qtd Paginas= 68470
351 - https://

476 - https://scryfall.com/sets/c16/de  Qtd Paginas= 96106
477 - https://scryfall.com/sets/c16/it  Qtd Paginas= 96510
478 - https://scryfall.com/sets/c16/pt  Qtd Paginas= 96914
479 - https://scryfall.com/sets/c16/ja  Qtd Paginas= 97318
480 - https://scryfall.com/sets/c16/zhs  Qtd Paginas= 97722
481 - https://scryfall.com/sets/tc16  Qtd Paginas= 97796
482 - https://scryfall.com/sets/oc16  Qtd Paginas= 97851
483 - https://scryfall.com/sets/ps16  Qtd Paginas= 97906
484 - https://scryfall.com/sets/kld  Qtd Paginas= 98230
485 - https://scryfall.com/sets/kld/es  Qtd Paginas= 98554
486 - https://scryfall.com/sets/kld/fr  Qtd Paginas= 98881
487 - https://scryfall.com/sets/kld/de  Qtd Paginas= 99208
488 - https://scryfall.com/sets/kld/it  Qtd Paginas= 99535
489 - https://scryfall.com/sets/kld/pt  Qtd Paginas= 99862
490 - https://scryfall.com/sets/kld/ja  Qtd Paginas= 100189
491 - https://scryfall.com/sets/kld/ko  Qtd Paginas= 100516
492 - https://scryfall.com/sets/kld/ru  Qtd Paginas= 100843
49

615 - https://scryfall.com/sets/dtk/de  Qtd Paginas= 128717
616 - https://scryfall.com/sets/dtk/it  Qtd Paginas= 129034
617 - https://scryfall.com/sets/dtk/pt  Qtd Paginas= 129351
618 - https://scryfall.com/sets/dtk/ja  Qtd Paginas= 129668
619 - https://scryfall.com/sets/dtk/ko  Qtd Paginas= 129985
620 - https://scryfall.com/sets/dtk/ru  Qtd Paginas= 130302
621 - https://scryfall.com/sets/dtk/zhs  Qtd Paginas= 130619
622 - https://scryfall.com/sets/dtk/zht  Qtd Paginas= 130936
623 - https://scryfall.com/sets/pdtk  Qtd Paginas= 131038
624 - https://scryfall.com/sets/tdtk  Qtd Paginas= 131096
625 - https://scryfall.com/sets/ptkdf  Qtd Paginas= 131150
626 - https://scryfall.com/sets/ddo  Qtd Paginas= 131267
627 - https://scryfall.com/sets/ddo/ja  Qtd Paginas= 131382
628 - https://scryfall.com/sets/frf  Qtd Paginas= 131620
629 - https://scryfall.com/sets/frf/es  Qtd Paginas= 131855
630 - https://scryfall.com/sets/frf/fr  Qtd Paginas= 132093
631 - https://scryfall.com/sets/frf/de  Qtd Pagin

754 - https://scryfall.com/sets/c13/it  Qtd Paginas= 156059
755 - https://scryfall.com/sets/c13/ja  Qtd Paginas= 156467
756 - https://scryfall.com/sets/oc13  Qtd Paginas= 156535
757 - https://scryfall.com/sets/ths  Qtd Paginas= 156834
758 - https://scryfall.com/sets/ths/es  Qtd Paginas= 157133
759 - https://scryfall.com/sets/ths/fr  Qtd Paginas= 157435
760 - https://scryfall.com/sets/ths/de  Qtd Paginas= 157737
761 - https://scryfall.com/sets/ths/it  Qtd Paginas= 158039
762 - https://scryfall.com/sets/ths/pt  Qtd Paginas= 158341
763 - https://scryfall.com/sets/ths/ja  Qtd Paginas= 158643
764 - https://scryfall.com/sets/ths/ko  Qtd Paginas= 158945
765 - https://scryfall.com/sets/ths/ru  Qtd Paginas= 159247
766 - https://scryfall.com/sets/ths/zhs  Qtd Paginas= 159549
767 - https://scryfall.com/sets/ths/zht  Qtd Paginas= 159851
768 - https://scryfall.com/sets/pths  Qtd Paginas= 159914
769 - https://scryfall.com/sets/tths  Qtd Paginas= 159975
770 - https://scryfall.com/sets/thp1  Qtd Pagin

893 - https://scryfall.com/sets/dka/pt  Qtd Paginas= 185138
894 - https://scryfall.com/sets/dka/ja  Qtd Paginas= 185349
895 - https://scryfall.com/sets/dka/ko  Qtd Paginas= 185560
896 - https://scryfall.com/sets/dka/ru  Qtd Paginas= 185771
897 - https://scryfall.com/sets/dka/zhs  Qtd Paginas= 185982
898 - https://scryfall.com/sets/dka/zht  Qtd Paginas= 186193
899 - https://scryfall.com/sets/pdka  Qtd Paginas= 186251
900 - https://scryfall.com/sets/tdka  Qtd Paginas= 186305
901 - https://scryfall.com/sets/pidw  Qtd Paginas= 186364
902 - https://scryfall.com/sets/pwp12  Qtd Paginas= 186417
903 - https://scryfall.com/sets/pdp12  Qtd Paginas= 186470
904 - https://scryfall.com/sets/l12  Qtd Paginas= 186522
905 - https://scryfall.com/sets/j12  Qtd Paginas= 186581
906 - https://scryfall.com/sets/f12  Qtd Paginas= 186644
907 - https://scryfall.com/sets/pd3  Qtd Paginas= 186724
908 - https://scryfall.com/sets/isd  Qtd Paginas= 187038
909 - https://scryfall.com/sets/isd/es  Qtd Paginas= 187352
9

### Site Add depois dos anteriores
* http://www.mtgmintcard.com/

In [41]:
inicio = time.time()
CardsCrawler(urlparse(Dominio[8]))
fim = time.time()
tempo = fim - inicio
print('Tempo do Site: ',tempo, 'Site: ', Dominio[8])
TempoDominio.append(dict(zip(['Dominio','Tempo'],[Dominio[8],tempo])))

Dominio: http://www.mtgmintcard.com/
1 - http://www.mtgmintcard.com/  Qtd Paginas= 553
2 - https://www.mtgmintcard.com/create-account  Qtd Paginas= 1020
3 - http://www.mtgmintcard.com/mtg/pdb  Qtd Paginas= 1518
4 - http://www.mtgmintcard.com/mtg/card-lots  Qtd Paginas= 2026
5 - http://www.mtgmintcard.com/accessories  Qtd Paginas= 2552
6 - http://www.mtgmintcard.com/buylist  Qtd Paginas= 3024
7 - http://www.mtgmintcard.com/articles  Qtd Paginas= 3586
8 - http://www.mtgmintcard.com/#standard  Qtd Paginas= 4138
9 - http://www.mtgmintcard.com/#modern  Qtd Paginas= 4690
10 - http://www.mtgmintcard.com/#legacy  Qtd Paginas= 5242
11 - http://www.mtgmintcard.com/#oldSchool  Qtd Paginas= 5794
12 - http://www.mtgmintcard.com/#commander  Qtd Paginas= 6346
13 - http://www.mtgmintcard.com/#mastersSets  Qtd Paginas= 6898
14 - http://www.mtgmintcard.com/#masterpieces  Qtd Paginas= 7450
15 - http://www.mtgmintcard.com/#promo  Qtd Paginas= 8002
16 - http://www.mtgmintcard.com/#ftv  Qtd Paginas= 8554
17

123 - http://www.mtgmintcard.com/mtg/singles/2ed  Qtd Paginas= 73505
124 - http://www.mtgmintcard.com/mtg/singles/leb  Qtd Paginas= 74081
125 - http://www.mtgmintcard.com/mtg/singles/lea  Qtd Paginas= 74658
126 - http://www.mtgmintcard.com/mtg/singles/c19  Qtd Paginas= 75271
127 - http://www.mtgmintcard.com/mtg/singles/c18  Qtd Paginas= 75885
128 - http://www.mtgmintcard.com/mtg/singles/cm2  Qtd Paginas= 76498
129 - http://www.mtgmintcard.com/mtg/singles/c17  Qtd Paginas= 77106
130 - http://www.mtgmintcard.com/mtg/singles/cma  Qtd Paginas= 77713
131 - http://www.mtgmintcard.com/mtg/singles/c16  Qtd Paginas= 78320
132 - http://www.mtgmintcard.com/mtg/singles/c15  Qtd Paginas= 78936
133 - http://www.mtgmintcard.com/mtg/singles/c14  Qtd Paginas= 79551
134 - http://www.mtgmintcard.com/mtg/singles/c13  Qtd Paginas= 80161
135 - http://www.mtgmintcard.com/mtg/singles/cm1  Qtd Paginas= 80718
136 - http://www.mtgmintcard.com/mtg/singles/com  Qtd Paginas= 81330
137 - http://www.mtgmintcard.com/m

241 - http://www.mtgmintcard.com/jump-start-card-lot-series  Qtd Paginas= 140252
242 - http://www.mtgmintcard.com/#carousel-example-generic  Qtd Paginas= 140804
243 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ra=4  Qtd Paginas= 141371
244 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ra=3  Qtd Paginas= 141985
245 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ra=2  Qtd Paginas= 142599
246 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ra=1  Qtd Paginas= 143213
247 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg?fil_ra=4  Qtd Paginas= 143783
248 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg?fil_ra=3  Qtd Paginas= 144399
249 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg?fil_ra=2  Qtd Paginas= 145015
250 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg?fil_ra=1  Qtd Paginas= 145631
251 - http://www.mtgmintcard.com/mtg/singles/war/eng-reg?fil_ra=4  Qtd Paginas= 146200
252 - http://www.mtgmintcard.com/mtg/singles/war/eng-reg

329 - http://www.mtgmintcard.com/mtg/card-lots/eld/eng/throne-of-eldraine-eld-complete-set-and-token-without-mythics  Qtd Paginas= 185555
330 - http://www.mtgmintcard.com/mtg/card-lots/eld/eng/60-foil-throne-of-eldraine-eld-common-card-lot  Qtd Paginas= 186032
331 - http://www.mtgmintcard.com/mtg/card-lots/eld/eng/40-foil-throne-of-eldraine-eld-uncommon-card-lot  Qtd Paginas= 186509
332 - http://www.mtgmintcard.com/mtg/card-lots/eld/eng/100-throne-of-eldraine-eld-rare-card-lot  Qtd Paginas= 186986
333 - http://www.mtgmintcard.com/mtg/card-lots/eld/eng/10-foil-throne-of-eldraine-eld-rare-card-lot  Qtd Paginas= 187463
334 - http://www.mtgmintcard.com/mtg/card-lots/m20/eng/core-set-2020-m20-uncommon-and-common-set-x4  Qtd Paginas= 187941
335 - http://www.mtgmintcard.com/mtg/card-lots/m20/eng/core-set-2020-m20-complete-set-with-mythics-x4-and-token-x4  Qtd Paginas= 188418
336 - http://www.mtgmintcard.com/mtg/card-lots/m20/eng/core-set-2020-m20-complete-set-with-mythics-and-token  Qtd Pagin

392 - http://www.mtgmintcard.com/accessories/card-sleeves/dragon-shield/dragon-shield-60-standard-deck-protector-sleeves-matte-night-blue-at-11242  Qtd Paginas= 215899
393 - http://www.mtgmintcard.com/accessories/card-sleeves/dragon-shield/dragon-shield-60-standard-deck-protector-sleeves-night-blue-at-10742  Qtd Paginas= 216375
394 - http://www.mtgmintcard.com/accessories/card-sleeves/dragon-shield/dragon-shield-100-standard-deck-protector-sleeves-matte-night-blue-at-11042  Qtd Paginas= 216851
395 - http://www.mtgmintcard.com/accessories/card-sleeves?page=1  Qtd Paginas= 217376
396 - http://www.mtgmintcard.com/accessories/card-sleeves?page=2  Qtd Paginas= 217903
397 - http://www.mtgmintcard.com/accessories/card-sleeves?page=3  Qtd Paginas= 218431
398 - http://www.mtgmintcard.com/accessories/card-sleeves?page=4  Qtd Paginas= 218960
399 - http://www.mtgmintcard.com/accessories/card-sleeves?page=5  Qtd Paginas= 219490
400 - http://www.mtgmintcard.com/accessories/card-sleeves?page=6  Qtd P

...

688 - http://www.mtgmintcard.com/mtg/singles/search?action=advanced_search&ed=Standard+Format&mo_1=1&mo_2=1&mo_3=1&lg_1=1&lg_2=1&lg_3=1&cc_1=1&cc_2=1&cc_4=1&cc_6=1&cc_7=1&cc_3=1&cc_8=1&cf=0&ct=-1&t2=&pf=-1&pt=99&tf=-1&tt=99&ra_4=1&ra_3=1&ra_2=1&ra_1=1&ra_6=1&rt=&prf=0.00&prt=-1&is=1&fil_col=8  Qtd Paginas= 381684
689 - http://www.mtgmintcard.com/mtg/singles/search?action=advanced_search&ed=Standard+Format&mo_1=1&mo_2=1&mo_3=1&lg_1=1&lg_2=1&lg_3=1&cc_1=1&cc_2=1&cc_4=1&cc_6=1&cc_7=1&cc_3=1&cc_8=1&cf=0&ct=-1&t2=&pf=-1&pt=99&tf=-1&tt=99&ra_4=1&ra_3=1&ra_2=1&ra_1=1&ra_6=1&rt=&prf=0.00&prt=-1&is=1&fil_ty=1  Qtd Paginas= 382304
690 - http://www.mtgmintcard.com/mtg/singles/search?action=advanced_search&ed=Standard+Format&mo_1=1&mo_2=1&mo_3=1&lg_1=1&lg_2=1&lg_3=1&cc_1=1&cc_2=1&cc_4=1&cc_6=1&cc_7=1&cc_3=1&cc_8=1&cf=0&ct=-1&t2=&pf=-1&pt=99&tf=-1&tt=99&ra_4=1&ra_3=1&ra_2=1&ra_1=1&ra_6=1&rt=&prf=0.00&prt=-1&is=1&fil_ty=3  Qtd Paginas= 382922
691 - http://www.mtgmintcard.com/mtg/singles/search?actio

732 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg/faeburrow-elder-extended-art  Qtd Paginas= 405376
733 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg/murderous-rider-showcase  Qtd Paginas= 405853
734 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg/the-cauldron-of-eternity-extended-art  Qtd Paginas= 406330
735 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=A  Qtd Paginas= 406381
736 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=B  Qtd Paginas= 406909
737 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=C  Qtd Paginas= 407449
738 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=D  Qtd Paginas= 407971
739 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=E  Qtd Paginas= 408499
740 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=F  Qtd Paginas= 409045
741 - http://www.mtgmintcard.com/mtg/singles/eldv/eng-reg?fil_alph=G  Qtd Paginas= 409576
742 - http://www.mtgmintcard.com/mtg/sing

821 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ty=3  Qtd Paginas= 453508
822 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ty=4  Qtd Paginas= 454089
823 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ty=5  Qtd Paginas= 454700
824 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ty=6  Qtd Paginas= 455314
825 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ty=13  Qtd Paginas= 455907
826 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?fil_ra=6  Qtd Paginas= 456487
827 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?view=2  Qtd Paginas= 457188
828 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?sort_by=name&sort=asc  Qtd Paginas= 457802
829 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?sort_by=cost&sort=asc  Qtd Paginas= 458416
830 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg?sort_by=price&sort=asc  Qtd Paginas= 459030
831 - http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/brazen-borrower  Qtd Paginas= 4594

910 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/veil-of-summer  Qtd Paginas= 502647
911 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/cavalier-of-thorns  Qtd Paginas= 503204
912 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/chandra-acolyte-of-flame  Qtd Paginas= 503763
913 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/leyline-of-anticipation  Qtd Paginas= 504328
914 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/lotus-field  Qtd Paginas= 504885
915 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/mu-yanling-sky-dancer  Qtd Paginas= 505442
916 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/ajani-strength-of-the-pride  Qtd Paginas= 505999
917 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/shifting-ceratops  Qtd Paginas= 506556
918 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg?page=2&scroll=1  Qtd Paginas= 507169
919 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/vivien-arkbow-ranger  Qtd Paginas= 507729
920 - http://www.mtg

998 - http://www.mtgmintcard.com/mtg/singles/war/eng-reg/jace-wielder-of-mysteries  Qtd Paginas= 552280
999 - http://www.mtgmintcard.com/mtg/singles/war/eng-reg/narset-parter-of-veils  Qtd Paginas= 552835
1000 - http://www.mtgmintcard.com/mtg/singles/war/eng-reg/bolass-citadel  Qtd Paginas= 553394
Tempo do Site:  4624.374035835266 Site:  http://www.mtgmintcard.com/


### Tabela com informações do Crawler Baseline.
* Foi reescrito só com os valores do ultimo site(mtgmintcard), porem ja tinhamos salvado um arquivo csv com os outros.

In [42]:
Dados

[{'Número': 1,
  'Link': 'http://www.mtgmintcard.com/',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 2,
  'Link': 'https://www.mtgmintcard.com/create-account',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 3,
  'Link': 'http://www.mtgmintcard.com/accessories',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 1,
  'Link': 'http://www.mtgmintcard.com/',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 2,
  'Link': 'https://www.mtgmintcard.com/create-account',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 3,
  'Link': 'http://www.mtgmintcard.com/mtg/pdb',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 4,
  'Link': 'http://www.mtgmintcard.com/mtg/card-lots',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 5,
  'Link': 'http://www.mtgmintcard.com/accessories',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 6,
  'Link': 'http://www.mtgmintcard.com/buylist',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 7,
  'Link': 'http://www.mtgmintcard.com/articles',
  'Site': 'www.mtgmintcard.com'},
 {'Número': 8

In [64]:
data = pd.DataFrame(Dados,columns=['Número','Link','Site'])
data.head()

,Número,Link,Site
0,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
1,2,https://www.mtgmintcard.com/create-account,www.mtgmintcard.com
2,3,http://www.mtgmintcard.com/accessories,www.mtgmintcard.com
3,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
4,2,https://www.mtgmintcard.com/create-account,www.mtgmintcard.com


### Arquivos Dados - csv

In [63]:
data.to_csv('CrawlerBaselineMtgMintCardInfo.csv',index_label=False)
data.head()

,Número,Link,Site
0,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
1,2,https://www.mtgmintcard.com/create-account,www.mtgmintcard.com
2,3,http://www.mtgmintcard.com/accessories,www.mtgmintcard.com
3,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
4,2,https://www.mtgmintcard.com/create-account,www.mtgmintcard.com


In [18]:
# Data dos anteriores ja foi escrita em arquivo csv
#data.to_csv('CrawlerBaselineInfo.csv',index_label=False)
#data.head()

,Número,Link,Site
0,1,https://www.capefeargames.com/,www.capefeargames.com
1,2,https://www.capefeargames.com/buylist,www.capefeargames.com
2,3,https://www.capefeargames.com/faq__policies,www.capefeargames.com
3,4,https://www.capefeargames.com/specials,www.capefeargames.com
4,5,https://www.capefeargames.com/products/multi_s...,www.capefeargames.com


### Tempo de Download das pages dos Dominios

In [15]:
# Valores do For loop.
TempoDominio

[{'Dominio': 'https://www.capefeargames.com/', 'Tempo': 4408.895867109299},
 {'Dominio': 'https://www.capefeargames.com/', 'Tempo': 3963.3277723789215},
 {'Dominio': 'https://www.mtgotraders.com/', 'Tempo': 5226.134366989136},
 {'Dominio': 'https://www.wizardscupboard.com/', 'Tempo': 4735.487655639648},
 {'Dominio': 'https://abugames.com/', 'Tempo': 17353.36330461502},
 {'Dominio': 'https://www.cardkingdom.com/', 'Tempo': 2087.5765290260315},
 {'Dominio': 'http://www.starcitygames.com/', 'Tempo': 1519.5797073841095},
 {'Dominio': 'https://www.tcgplayer.com/', 'Tempo': 36.853039264678955},
 {'Dominio': 'https://scryfall.com/', 'Tempo': 2028.5180823802948}]

In [44]:
# Valor do ultimo link.
TempoDominio

[{'Dominio': 'http://www.mtgmintcard.com/', 'Tempo': 4624.374035835266}]

In [84]:
#Como a variavel TempoDominio foi reescrita, vamos reescrever ela novamente com os valores que ja tinhamoes com o novo.
tempoCapeFear = {'Dominio': 'https://www.capefeargames.com/', 'Tempo': int(3963.3277723789215)}
tempoMtgotraders = {'Dominio': 'https://www.mtgotraders.com/', 'Tempo': int(5226.134366989136)}
tempoWiza = {'Dominio': 'https://www.wizardscupboard.com/', 'Tempo': int(4735.487655639648)}
tempoAbu = {'Dominio': 'https://abugames.com/', 'Tempo': int(17353.36330461502)}
tempoCardKing = {'Dominio': 'https://www.cardkingdom.com/', 'Tempo': int(2087.5765290260315)}
tempoStart = {'Dominio': 'http://www.starcitygames.com/', 'Tempo': int(1519.5797073841095)}
tempoTcg = {'Dominio': 'https://www.tcgplayer.com/', 'Tempo': int(36.853039264678955)}
tempoScry = {'Dominio': 'https://scryfall.com/', 'Tempo': int(2028.5180823802948)}
tempoMtgmint = {'Dominio': 'http://www.mtgmintcard.com/', 'Tempo': int(4624.374035835266)}

TempoDominio = [tempoCapeFear,tempoMtgotraders,tempoAbu,tempoCardKing,tempoMtgmint,tempoScry,tempoStart,tempoWiza,tempoTcg]

In [85]:
TempoDominio

[{'Dominio': 'https://www.capefeargames.com/', 'Tempo': 3963},
 {'Dominio': 'https://www.mtgotraders.com/', 'Tempo': 5226},
 {'Dominio': 'https://abugames.com/', 'Tempo': 17353},
 {'Dominio': 'https://www.cardkingdom.com/', 'Tempo': 2087},
 {'Dominio': 'http://www.mtgmintcard.com/', 'Tempo': 4624},
 {'Dominio': 'https://scryfall.com/', 'Tempo': 2028},
 {'Dominio': 'http://www.starcitygames.com/', 'Tempo': 1519},
 {'Dominio': 'https://www.wizardscupboard.com/', 'Tempo': 4735},
 {'Dominio': 'https://www.tcgplayer.com/', 'Tempo': 36}]

In [86]:
timeData = pd.DataFrame(TempoDominio,columns=['Dominio','Tempo'])
#timeData["Tempo-Download"] = pd.to_numeric(timeData["Tempo-Download"])
timeData

,Dominio,Tempo
0,https://www.capefeargames.com/,3963
1,https://www.mtgotraders.com/,5226
2,https://abugames.com/,17353
3,https://www.cardkingdom.com/,2087
4,http://www.mtgmintcard.com/,4624
5,https://scryfall.com/,2028
6,http://www.starcitygames.com/,1519
7,https://www.wizardscupboard.com/,4735
8,https://www.tcgplayer.com/,36


### Média Tempo-Download

In [87]:
print('Média Tempo-Download em Segundos: ',timeData['Tempo'].mean())
print('Média Tempo-Download em Minutos: ',timeData['Tempo'].mean()/60)
print('Média Tempo-Download em Horas: ',(timeData['Tempo'].mean()/60)/60)

Média Tempo-Download em Segundos:  4619.0
Média Tempo-Download em Minutos:  76.98333333333333
Média Tempo-Download em Horas:  1.2830555555555556


### Dataframe em Minutos

In [88]:
# Convertendo para minutos
timeData['Tempo'] = timeData['Tempo'].apply(lambda x: x / 60)

In [89]:
timeData

,Dominio,Tempo
0,https://www.capefeargames.com/,66.050000
1,https://www.mtgotraders.com/,87.100000
2,https://abugames.com/,289.216667
3,https://www.cardkingdom.com/,34.783333
4,http://www.mtgmintcard.com/,77.066667
5,https://scryfall.com/,33.800000
6,http://www.starcitygames.com/,25.316667
7,https://www.wizardscupboard.com/,78.916667
8,https://www.tcgplayer.com/,0.600000


### DataFrame em Horas

In [90]:
# Convertendo para horas
timeData['Tempo'] = timeData['Tempo'].apply(lambda x: x / 60)
timeData

,Dominio,Tempo
0,https://www.capefeargames.com/,1.100833
1,https://www.mtgotraders.com/,1.451667
2,https://abugames.com/,4.820278
3,https://www.cardkingdom.com/,0.579722
4,http://www.mtgmintcard.com/,1.284444
5,https://scryfall.com/,0.563333
6,http://www.starcitygames.com/,0.421944
7,https://www.wizardscupboard.com/,1.315278
8,https://www.tcgplayer.com/,0.010000


### Tempo de Download - csv

In [92]:
# Tempo de download dos anteriores - tempo em horas
timeData.to_csv('CrawlerBaselineTime.csv',index_label=False)
timeData

,Dominio,Tempo
0,https://www.capefeargames.com/,1.100833
1,https://www.mtgotraders.com/,1.451667
2,https://abugames.com/,4.820278
3,https://www.cardkingdom.com/,0.579722
4,http://www.mtgmintcard.com/,1.284444
5,https://scryfall.com/,0.563333
6,http://www.starcitygames.com/,0.421944
7,https://www.wizardscupboard.com/,1.315278
8,https://www.tcgplayer.com/,0.010000
